In [1]:
import numpy as np
import pandas as pd
from astropy.io import ascii
from scipy import stats
from scipy.optimize import minimize,curve_fit
from scipy.stats import gaussian_kde as kde
np.random.seed(9620)

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
sns.set(style='ticks', context='talk')
plt.style.use("../paper.mplstyle")

from multiprocessing import Pool
from tqdm import tqdm

from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord, Galactic
from astropy import units as u

from warnings import filterwarnings
filterwarnings(action='ignore')

# load data

In [3]:
from astropy.table import Table
table = Table.read('../data/PyCasso/CALIFA_1.fits')
califa1 = table.to_pandas().set_index('CALIFA')

table = Table.read('../data/PyCasso/pycasso_integrated_main_zca6e.fits')
pycasso = table.to_pandas().set_index('califaID')

In [11]:
califa1

,Name,Type,logM*,logmu*0,lognmu*HLR,AV0,AVHLR,logage0,logageHLR,logZM0,logZ0MHLR,HLR,HMR,C
CALIFA,,,,,,,,,,,,,,
1,b'IC 5376 ',b' 2 A ',10.60,3.87,2.41,0.42,0.53,9.97,9.38,0.25,-0.17,3883,2076,3.3
2,b'UGC 00005 ',b' 3 A ',10.84,3.29,2.23,0.63,0.41,9.51,8.88,-0.03,-0.04,6842,5146,2.1
3,b'NGC 7819 ',b' 4 A ',10.04,2.80,1.59,0.63,0.05,8.46,8.86,0.13,-0.27,5455,4057,2.1
4,b'UGC 00029 ',b'-1 A ',11.16,3.67,2.49,0.44,0.03,9.89,9.75,0.37,0.14,5400,4372,3.1
5,b'IC 1528 ',b' 3 AB',10.09,3.03,1.95,0.80,0.35,8.97,8.64,-0.22,-0.29,4348,3417,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,b'UGC 12864 ',b' 4 B ',9.92,2.73,1.61,0.46,0.24,8.76,8.48,-0.44,-0.18,5227,4619,2.4
936,b'MCG-01-01-012 ',b' 1 AB',10.62,3.37,2.45,0.77,0.20,9.90,9.59,-0.06,0.21,4977,4155,2.6
937,b'NGC 7800 ',b' 5 AB',8.98,2.30,1.64,0.37,0.37,8.02,8.23,-0.37,-0.47,1598,1639,2.4


In [27]:
# convert Hubble types into T-values for convenience
types = ['E','S0-','S0','S0+','S0a','Sa','Sab','Sb','Sbc','Sc','Scd','Sd','Sdm']
T_center = [-5,-3,-2,-1,0,1,2,3,4,5,6,7,8]

# CALIFA morphology uses a weird numerical system:
CALIFA_types = ['E','S0','Sa','Sb','Sbc','Sc','Sd']
CALIFA_numerical = [-1,0,1,2,3,4,5]

In [29]:
df = pd.DataFrame(columns=['mass','age_lw','age_mw','ttype'])
for idx in califa1.index.values:
    try:
        assert califa1.loc[idx,'Name'].decode("utf-8").replace(" ", "")==pycasso.loc[idx,'NED_name'].decode("utf-8")
        # morph -- convert the paper-unique system to the standard numerical t-type
        tmp = califa1.loc[idx,'Type'].decode("utf-8")
        numerical = np.fromstring(tmp,sep=" ")[0]
        Hubble_type = CALIFA_types[np.where(np.asarray(CALIFA_numerical)==numerical)[0][0]]
        ttype = T_center[np.where(np.asarray(types)==Hubble_type)[0][0]]

        # mass
        mass = np.log10(pycasso.loc[idx,'M_star'])

        # age
        age_lw = 10**pycasso.loc[idx,'log_age_flux']/1e9
        age_mw = 10**pycasso.loc[idx,'log_age_mass']/1e9

        df.loc[idx] = [mass,age_lw,age_mw,ttype]
    except:
        continue

In [44]:
df.index = df.index.rename('califa_id')
df.to_csv('CALIFA_ttype.csv',index=True)

In [45]:
df

,mass,age_lw,age_mw,ttype
califa_id,,,,
1,10.504722,2.183996,6.118846,3.0
2,10.911792,1.073397,6.749292,4.0
3,10.220843,0.892115,6.364300,5.0
5,10.178084,0.732614,4.428360,4.0
7,10.926013,3.755816,8.962501,1.0
...,...,...,...,...
930,9.658766,0.142450,3.302069,5.0
933,10.564807,1.250340,5.764875,1.0
934,9.432574,0.711195,1.985418,4.0
